## установка библиотек, подгрузка API ключа, предобработка входных данных по правилам создателей TimeGPT

## импорт

In [1]:
!pip install nixtlats -q # установка модели TimeGPT-1
# метрики
!pip install sktime -q
from sktime.forecasting.model_selection import temporal_train_test_split # для разделения временных рядов на тренировочные и тестовые наборы.
from sktime.performance_metrics.forecasting import  MeanAbsolutePercentageError
smape = MeanAbsolutePercentageError(symmetric = True)#метрика sMAP
from sklearn.metrics import mean_squared_error,mean_absolute_error #метрики
from sklearn.model_selection import TimeSeriesSplit #для раздления набора данных
import pandas as pd
from prettytable import PrettyTable # для красоты и порядка

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 659.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.1/130.1 kB 13.0 MB/s eta 0:00:00


In [111]:
!pip install --upgrade gdown #https://github.com/wkentaro/gdown/issues/333
!git clone https://github.com/zhouhaoyi/ETDataset # офицальный репозиторий температуры машинного масла
!gdown --fuzzy https://drive.google.com/file/d/1nXdMIJ7K201Bx3IBGNiaNFQ6FzeDEzIr/view?usp=drive_link
!gdown --fuzzy https://drive.google.com/file/d/1rN79CxW3Vldp-WDuSoG0bKq9tYQR79UK/view?usp=drive_link
!gdown --fuzzy https://drive.google.com/file/d/1WMKg7KevVEEd9jrfLG8mcpOequZMbjlM/view?usp=drive_link
!unzip /content/weather.zip
!unzip /content/exchange_rate.zip
!unzip /content/illness.zip
# новыt наборы данных
!gdown --fuzzy https://drive.google.com/file/d/1-Jy7C9Dh9QJF_yWKf2hQKUia56aQXBJ0/view?usp=drive_link -q
!gdown --fuzzy https://drive.google.com/file/d/1lPZhTcJVirbwhN6EpppcmpZzbDcBn4rx/view?usp=drive_link -q

fatal: destination path 'ETDataset' already exists and is not an empty directory.
Downloading...
From: https://drive.google.com/uc?id=1nXdMIJ7K201Bx3IBGNiaNFQ6FzeDEzIr
To: /content/weather.zip
100% 2.29M/2.29M [00:00<00:00, 94.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rN79CxW3Vldp-WDuSoG0bKq9tYQR79UK
To: /content/exchange_rate.zip
100% 194k/194k [00:00<00:00, 109MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WMKg7KevVEEd9jrfLG8mcpOequZMbjlM
To: /content/illness.zip
100% 26.0k/26.0k [00:00<00:00, 39.2MB/s]
Archive:  /content/weather.zip
replace weather/weather.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/exchange_rate.zip
replace exchange_rate/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/illness.zip
replace illness/national_illness.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
from google.colab import files
uploaded = files.upload() # подгрузка токена в виртуальную среду

Saving diplom.json to diplom.json


In [5]:
import json
with open('diplom.json') as json_file:
    data = json.load(json_file)
    api_key = data['key'] # передаём в переменную api ключ

In [6]:
from nixtlats import TimeGPT # импортируем некоторые необходимые библиотеки

timegpt = TimeGPT(token = api_key) # проводим идентификацию ключа

/usr/local/lib/python3.10/dist-packages/nixtlats/__init__.py:5: FutureWarning: This package is deprecated, please install nixtla instead.
  warnings.warn("This package is deprecated, please install nixtla instead.", category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/nixtlats/nixtla_client.py:53: FutureWarning: `'token'` is deprecated; use `'api_key'` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/nixtlats/nixtla_client.py:1605: FutureWarning: Class `TimeGPT` is deprecated; use `NixtlaClient` instead.
  warnings.warn(


In [67]:
ETTh1 = pd.read_csv('/content/ETDataset/ETT-small/ETTh1.csv')
ETTh2 = pd.read_csv('/content/ETDataset/ETT-small/ETTh2.csv')
ETTm1 = pd.read_csv('/content/ETDataset/ETT-small/ETTm1.csv')
ETTm2 = pd.read_csv('/content/ETDataset/ETT-small/ETTm2.csv')
exchange_rate = pd.read_csv('/content/exchange_rate/exchange_rate.csv')
illness = pd.read_csv('/content/illness/national_illness.csv')
weather = pd.read_csv('/content/weather/weather.csv')
de_small = pd.read_csv('/content/de_small.csv')
de_big = pd.read_csv('/content/de_big.csv')

In [21]:

def trim_dataset(dataset, num_rows=1440):
    """
    Отрезает последние num_rows строк от датасета.

    Параметры:
    dataset (pandas.DataFrame) - исходный датасет
    num_rows (int) - количество строк, которые нужно отрезать (по умолчанию 1440)

    Возвращает:
    pandas.DataFrame - обрезанный датасет
    """
    total_rows = len(dataset)
    start_index = total_rows - num_rows

    if start_index < 0:
        raise ValueError("Количество строк в датасете меньше, чем требуемое число строк для отрезания.")

    return dataset.iloc[start_index:]

In [68]:
# берём толкьо часть набора данных, последние 720 точек или 60 на просмотр и 720 или 60 на прогнозирование
ETTh1 = trim_dataset(ETTh1, 1440)
ETTh2 = trim_dataset(ETTh2, 1440)
ETTm1 = trim_dataset(ETTm1, 1440)
ETTm2 = trim_dataset(ETTm2, 1440)
exchange_rate = trim_dataset(exchange_rate, 1440)
weather = trim_dataset(weather, 1440)
de_big = trim_dataset(de_big, 1440)
de_small = trim_dataset(de_small, 120)
illness = trim_dataset(illness, 120)

In [69]:
def TimeGPT_func_prep(df, ds, y):
  '''обработка наборов данных по правилам Nixtla'''
  df = df.rename(columns={ds: 'ds', y: 'y'}) # по правилам Nixtla
  df['ds'] = pd.to_datetime(df['ds'])
  df.index.name = 'unique_id' #
  df.reset_index(inplace=True) #
  df['unique_id'] = 1
  last_column = df.iloc[:, -1] # копируем последний столбец
  df = df.iloc[:, :-1] # удаляем последний столбец из исходного DataFrame
  df.insert(2, 'y', last_column) # вставляем скопированный столбец на вторую позицию (индекс 1)
  return df


In [70]:
df_list = [ETTh1, ETTh2, ETTm1, ETTm2,  exchange_rate, illness, weather, de_small, de_big]
for i, df in enumerate(df_list):
    if i == 0:
        ETTh1 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 1:
        ETTh2 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 2:
        ETTm1 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 3:
        ETTm2 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 4:
        exchange_rate = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 5:
        illness = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 6:
        weather = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 7:
        de_small = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 8:
        de_big = TimeGPT_func_prep(df, 'date', 'OT')

In [42]:
ETTh1[['ds', 'unique_id', 'y']]

,ds,unique_id,y
0,2018-04-27 20:00:00,1,11.678
1,2018-04-27 21:00:00,1,11.326
2,2018-04-27 22:00:00,1,11.044
3,2018-04-27 23:00:00,1,10.974
4,2018-04-28 00:00:00,1,10.833
...,...,...,...
1435,2018-06-26 15:00:00,1,10.904
1436,2018-06-26 16:00:00,1,11.044
1437,2018-06-26 17:00:00,1,10.271
1438,2018-06-26 18:00:00,1,9.778


In [71]:
def train_test_func(df):
    train_size = int(len(df) * 0.5)
    test_size = len(df) - train_size
    X_train, X_test = df[0:train_size], df[train_size:len(df)]
    return X_train, X_test

# Создание словаря для хранения результатов
split_data = {}

# Применение функции к каждому датасету из списка df_list
for df_name, df in zip(['ETTh1', 'ETTh2', 'ETTm1', 'ETTm2', 'exchange_rate', 'illness', 'weather','de_small', 'de_big'], df_list):
    X_train, X_test = train_test_func(df)
    split_data[f'{df_name}_train'] = X_train
    split_data[f'{df_name}_test'] = X_test

# Вывод размеров полученных выборок для каждого датасета
for key, value in split_data.items():
    print(f"Размер {key}: {len(value)}")


Размер ETTh1_train: 720
Размер ETTh1_test: 720
Размер ETTh2_train: 720
Размер ETTh2_test: 720
Размер ETTm1_train: 720
Размер ETTm1_test: 720
Размер ETTm2_train: 720
Размер ETTm2_test: 720
Размер exchange_rate_train: 720
Размер exchange_rate_test: 720
Размер illness_train: 60
Размер illness_test: 60
Размер weather_train: 720
Размер weather_test: 720
Размер de_small_train: 60
Размер de_small_test: 60
Размер de_big_train: 720
Размер de_big_test: 720


In [72]:
ETTh1_train, ETTh1_test = train_test_func(ETTh1[['ds','y','unique_id']])
ETTh2_train, ETTh2_test = train_test_func(ETTh2[['ds','y','unique_id']])
ETTm1_train, ETTm1_test = train_test_func(ETTm1[['ds','y','unique_id']])
ETTm2_train, ETTm2_test = train_test_func(ETTm2[['ds','y','unique_id']])
exchange_rate_train, exchange_rate_test = train_test_func(exchange_rate[['ds','y','unique_id']])
illness_train, illness_test = train_test_func(illness[['ds','y','unique_id']])
weather_train, weather_test = train_test_func(weather[['ds','y','unique_id']])
de_small_train, de_small_test = train_test_func(de_small[['ds','y','unique_id']])
de_big_train, de_big_test = train_test_func(de_big[['ds','y','unique_id']])

## прогнозирование без дообучения

#### ETTh1

 любыми способами, не получилось добиться от TimeGPT прогнозирвоание многомерного временного ряда или прогнозирование с использованием ковариант
 он всё время выполнял одномерный прогноз

In [52]:
%%time
df = timegpt.forecast(df=ETTh1_train,h=720, model='timegpt-1-long-horizon', freq='h') # любыми способоами, у меня не получилось добиться от
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTh1.csv")

CPU times: user 111 ms, sys: 4.42 ms, total: 115 ms
Wall time: 3.02 s


In [53]:
step_forecasting = [24, 96, 192, 336, 720]

In [54]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTh1_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('zero-shot ETTh1'.center(len(table.field_names) * 11))
print(table)

         zero-shot ETTh1         
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.269 | 0.098 |
|       96       | 0.468 | 0.379 |
|      192       | 0.626 | 0.659 |
|      336       | 0.909 | 1.859 |
|      720       | 1.291 |  3.18 |
+----------------+-------+-------+


#### ETTh2

In [55]:
%%time
df = timegpt.forecast(df=ETTh2_train,h=720, model='timegpt-1-long-horizon',  freq='h') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTh2.csv")

CPU times: user 76.8 ms, sys: 7.55 ms, total: 84.3 ms
Wall time: 2.61 s


In [56]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTh2_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('zero-shot ETTh2'.center(len(table.field_names) * 11))
print(table)

         zero-shot ETTh2         
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       | 1.204 | 1.864  |
|       96       | 1.872 | 4.238  |
|      192       | 2.239 | 8.212  |
|      336       | 2.844 | 13.414 |
|      720       | 4.094 | 26.748 |
+----------------+-------+--------+


#### ETTm1

In [57]:
%%time
df = timegpt.forecast(df=ETTm1_train,h=720, model='timegpt-1-long-horizon',freq='15min') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTm1.csv")

CPU times: user 80 ms, sys: 1.91 ms, total: 81.9 ms
Wall time: 2.62 s


на всём наборе данных лучше

In [58]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTm1_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot ETTm1'.center(len(table.field_names) * 11))
print(table)

         zero-shot ETTm1         
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       | 0.468 | 0.803  |
|       96       | 2.859 | 10.738 |
|      192       | 2.278 | 6.947  |
|      336       | 2.555 | 9.191  |
|      720       | 2.148 | 6.977  |
+----------------+-------+--------+


#### ETTm2

In [59]:
%%time
df = timegpt.forecast(df=ETTm2_train,h=720, model='timegpt-1-long-horizon',freq='15min') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTm2.csv")

CPU times: user 80.9 ms, sys: 4.06 ms, total: 85 ms
Wall time: 2.64 s


на всём наборе данных лучше

In [60]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTm2_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot ETTm2'.center(len(table.field_names) * 11))
print(table)

         zero-shot ETTm2         
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       | 1.691 |  3.3   |
|       96       | 5.778 | 42.362 |
|      192       | 5.229 | 36.451 |
|      336       | 5.566 | 43.358 |
|      720       | 5.633 | 44.503 |
+----------------+-------+--------+


#### exchange_rate

In [61]:
%%time
df = timegpt.forecast(df=exchange_rate_train, h=720, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("exchange_rate.csv")

CPU times: user 84.5 ms, sys: 4.87 ms, total: 89.3 ms
Wall time: 3.82 s


In [62]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = exchange_rate_train['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot exchange_rate'.center(len(table.field_names) * 11))
print(table)

     zero-shot exchange_rate     
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.054 | 0.003 |
|       96       | 0.051 | 0.003 |
|      192       | 0.038 | 0.002 |
|      336       | 0.028 | 0.001 |
|      720       | 0.031 | 0.001 |
+----------------+-------+-------+


#### weather

In [73]:
%%time
df = timegpt.forecast(df=weather_train, h=720, model='timegpt-1-long-horizon',freq='10min') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("weather2.csv")

CPU times: user 87.2 ms, sys: 3.74 ms, total: 90.9 ms
Wall time: 2.69 s


In [74]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = weather_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot weather'.center(len(table.field_names) * 11))
print(table)

        zero-shot weather        
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       | 1.174 | 1.881  |
|       96       |  1.44 | 2.583  |
|      192       | 1.656 | 3.601  |
|      336       | 3.735 | 32.456 |
|      720       | 3.356 | 23.352 |
+----------------+-------+--------+


In [85]:
%%time
df = timegpt.forecast(df=de_big_train, h=720,  freq='h' ) # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("Predict_de_big.csv")

CPU times: user 86.9 ms, sys: 5.41 ms, total: 92.3 ms
Wall time: 6.37 s


In [86]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = de_big_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot de_big'.center(len(table.field_names) * 11))
print(table)

         zero-shot de_big        
+----------------+----------+--------------+
| Шагов прогноза |   MAE    |     MSE      |
+----------------+----------+--------------+
|       24       | 762.589  |  748237.146  |
|       96       | 2205.256 | 7692203.216  |
|      192       | 2012.095 | 6417562.789  |
|      336       | 2406.51  | 9221813.379  |
|      720       | 2686.644 | 11925016.779 |
+----------------+----------+--------------+


In [ ]:
%%time
df = timegpt.forecast(df=de_big_train, h=720, model='timegpt-1-long-horizon',  freq='h' ) # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("predict_de_big.csv")

In [84]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = de_big_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot de_big'.center(len(table.field_names) * 11))
print(table)

         zero-shot de_big        
+----------------+----------+-------------+
| Шагов прогноза |   MAE    |     MSE     |
+----------------+----------+-------------+
|       24       | 1089.628 | 1830413.181 |
|       96       | 2384.361 | 8205610.814 |
|      192       | 1581.111 | 4664174.427 |
|      336       | 1543.019 | 4153943.007 |
|      720       | 1692.266 | 4549992.236 |
+----------------+----------+-------------+


#### illnes

In [90]:
step_forecasting_60 = [24,36,48,60]

In [96]:
%%time
df = timegpt.forecast(df=illness_train, h=60, freq='7D' ) # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ill.csv")

CPU times: user 59.8 ms, sys: 1.41 ms, total: 61.2 ms
Wall time: 2.96 s


In [97]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = illness_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting_60:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot illness'.center(len(table.field_names) * 11))
print(table)

        zero-shot illness        
+----------------+------------+------------------+
| Шагов прогноза |    MAE     |       MSE        |
+----------------+------------+------------------+
|       24       | 436029.115 | 265398343121.884 |
|       36       | 348180.479 | 188914661166.487 |
|       48       | 276043.422 | 143544794849.915 |
|       60       | 232876.181 | 116058057894.529 |
+----------------+------------+------------------+


наблюдаю абсолютную невозможность предсказать illness

In [98]:
%%time
df = timegpt.forecast(df=de_small_train, h=60, freq='h' ) # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("de_small_predict.csv")

CPU times: user 66.7 ms, sys: 4.89 ms, total: 71.6 ms
Wall time: 2.7 s


In [99]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = de_small_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting_60:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot de_small'.center(len(table.field_names) * 11))
print(table)

        zero-shot de_small       
+----------------+----------+--------------+
| Шагов прогноза |   MAE    |     MSE      |
+----------------+----------+--------------+
|       24       | 2463.146 | 10263491.96  |
|       36       | 3185.766 | 21475462.878 |
|       48       | 5817.295 | 66344264.754 |
|       60       | 6444.403 | 75617613.373 |
+----------------+----------+--------------+


## с дообучением на 200 точках
значительно больше 200 точек не получалось взять

In [135]:
ETTh1 = pd.read_csv('/content/ETDataset/ETT-small/ETTh1.csv')
ETTh2 = pd.read_csv('/content/ETDataset/ETT-small/ETTh2.csv')
ETTm1 = pd.read_csv('/content/ETDataset/ETT-small/ETTm1.csv')
ETTm2 = pd.read_csv('/content/ETDataset/ETT-small/ETTm2.csv')
exchange_rate = pd.read_csv('/content/exchange_rate/exchange_rate.csv')
illness = pd.read_csv('/content/illness/national_illness.csv')
weather = pd.read_csv('/content/weather/weather.csv')
de_small = pd.read_csv('/content/de_small.csv')
de_big = pd.read_csv('/content/de_big.csv')

In [136]:
# берём толкьо часть набора данных, последние 720 точек или 60 на просмотр и 720 или 60 на прогнозирование
ETTh1 = trim_dataset(ETTh1, 1940)
ETTh2 = trim_dataset(ETTh2, 1940)
ETTm1 = trim_dataset(ETTm1, 1940)
ETTm2 = trim_dataset(ETTm2, 1940)
exchange_rate = trim_dataset(exchange_rate, 1940)
weather = trim_dataset(weather, 1940)
de_big = trim_dataset(de_big, 1940)
de_small = trim_dataset(de_small, 420)
illness = trim_dataset(illness, 420)

In [137]:
df_list = [ETTh1, ETTh2, ETTm1, ETTm2,  exchange_rate, illness, weather, de_small, de_big]
for i, df in enumerate(df_list):
    if i == 0:
        ETTh1 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 1:
        ETTh2 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 2:
        ETTm1 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 3:
        ETTm2 = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 4:
        exchange_rate = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 5:
        illness = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 6:
        weather = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 7:
        de_small = TimeGPT_func_prep(df, 'date', 'OT')
    elif i == 8:
        de_big = TimeGPT_func_prep(df, 'date', 'OT')

In [138]:
def train_test_finetune_func(df):
    train_size = int(len(df) * 0.6)
    test_size = len(df) - train_size
    X_train, X_test = df[0:train_size], df[train_size:len(df)]
    return X_train, X_test

In [139]:
ETTh1_train, ETTh1_test = train_test_finetune_func(ETTh1[['ds','y','unique_id']])
ETTh2_train, ETTh2_test = train_test_finetune_func(ETTh2[['ds','y','unique_id']])
ETTm1_train, ETTm1_test = train_test_finetune_func(ETTm1[['ds','y','unique_id']])
ETTm2_train, ETTm2_test = train_test_finetune_func(ETTm2[['ds','y','unique_id']])
exchange_rate_train, exchange_rate_test = train_test_finetune_func(exchange_rate[['ds','y','unique_id']])
illness_train, illness_test = train_test_finetune_func(illness[['ds','y','unique_id']])
weather_train, weather_test = train_test_finetune_func(weather[['ds','y','unique_id']])
de_small_train, de_small_test = train_test_finetune_func(de_small[['ds','y','unique_id']])
de_big_train, de_big_test = train_test_finetune_func(de_big[['ds','y','unique_id']])

#### ETTh1 few200

In [140]:
%%time
df = timegpt.forecast(df=ETTh1_train,h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания

CPU times: user 224 ms, sys: 15 ms, total: 239 ms
Wall time: 23.8 s


In [ ]:
predict.to_csv("ETTh1_few200.csv")

на всём наборе в некоторых горизнотах чуть лучше

In [141]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTh1_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 ETTh1'.center(len(table.field_names) * 11))
print(table)

        few-shot 200 ETTh1       
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 1.969 |  5.09 |
|       96       | 1.118 | 2.065 |
|      192       | 1.422 | 2.796 |
|      336       | 1.701 | 3.961 |
|      720       | 1.794 | 4.731 |
+----------------+-------+-------+


#### ETTh2 few 200

In [142]:
%%time
df = timegpt.forecast(df=ETTh2_train, h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTh2_few200.csv")

CPU times: user 238 ms, sys: 20.6 ms, total: 259 ms
Wall time: 29.3 s


2.325 первый горизонт лучше на всём наборе данных

In [143]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTh2_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 ETTh2'.center(len(table.field_names) * 11))
print(table)

        few-shot 200 ETTh2       
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       | 2.978 | 12.547 |
|       96       | 2.325 | 7.927  |
|      192       | 3.271 | 16.022 |
|      336       | 4.791 | 36.218 |
|      720       | 6.655 | 67.737 |
+----------------+-------+--------+


### ETTm1 few 200

In [144]:
df = timegpt.forecast(df=ETTm1_train,h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTm1_few200.csv")

на всём наборе лучше результаты

In [ ]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTm1_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 ETTm1'.center(len(table.field_names) * 11))
print(table)

        few-shot 200 ETTm1       
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.504 | 0.323 |
|       96       | 0.748 | 0.697 |
|      192       | 0.868 |  1.02 |
|      336       | 0.963 | 1.388 |
|      720       | 0.965 | 1.324 |
+----------------+-------+-------+


### ETTm2 few 200

In [146]:
df = timegpt.forecast(df=ETTm2_train, h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("ETTm2_few200.csv")

на всём наборе данных лучше 192, 336, 720

In [147]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = ETTm2_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 ETTm2'.center(len(table.field_names) * 11))
print(table)

        few-shot 200 ETTm2       
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       | 0.926 | 1.028  |
|       96       | 2.794 | 17.27  |
|      192       | 4.923 | 35.948 |
|      336       | 4.661 | 30.311 |
|      720       | 5.972 | 51.048 |
+----------------+-------+--------+


### exchange_rate few 200

In [148]:
df = timegpt.forecast(df=exchange_rate_train, h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("exchange_rate_few200.csv")

In [ ]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = exchange_rate_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 exchange_rate'.center(len(table.field_names) * 11))
print(table)

        few-shot 200 ETTm2       
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.008 |  0.0  |
|       96       | 0.013 |  0.0  |
|      192       | 0.035 | 0.002 |
|      336       | 0.045 | 0.003 |
|      720       | 0.098 | 0.013 |
+----------------+-------+-------+


на всём наборе данных лучше

In [149]:
# Объявляем таблицу для красивого вывода результатов
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = exchange_rate_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
# Печатаем таблицу
print('few-shot 200 exchange_rate'.center(len(table.field_names) * 11))
print(table)

    few-shot 200 exchange_rate   
+----------------+-------+-------+
| Шагов прогноза |  MAE  |  MSE  |
+----------------+-------+-------+
|       24       | 0.016 |  0.0  |
|       96       | 0.062 | 0.005 |
|      192       | 0.075 | 0.007 |
|      336       | 0.123 | 0.019 |
|      720       | 0.148 | 0.025 |
+----------------+-------+-------+


### weather few 200

In [150]:
%%time
df = timegpt.forecast(df=weather_train, h=720, finetune_steps=200, model='timegpt-1-long-horizon',freq='10min') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("weather200.csv")

CPU times: user 225 ms, sys: 22.5 ms, total: 247 ms
Wall time: 23.8 s


In [151]:
table = PrettyTable()
table.field_names = ["Шагов прогноза", "MAE", "MSE"]
test = weather_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('zero-shot weather'.center(len(table.field_names) * 11))
print(table)

        zero-shot weather        
+----------------+-------+--------+
| Шагов прогноза |  MAE  |  MSE   |
+----------------+-------+--------+
|       24       | 4.487 | 24.472 |
|       96       | 2.621 | 11.138 |
|      192       | 2.456 | 9.613  |
|      336       | 3.843 | 22.107 |
|      720       | 5.979 | 52.801 |
+----------------+-------+--------+


### illness few 200

In [152]:
df = timegpt.forecast(df=illness_train, h=60, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("illness_few200.csv")

In [153]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = illness_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting_60:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('few-shot 200 illness'.center(len(table.field_names) * 11))
print(table)

       few-shot 200 illness      
+----------------+-----------+-----------------+
| Шагов прогноза |    MAE    |       MSE       |
+----------------+-----------+-----------------+
|       24       | 61135.174 |  5343712858.849 |
|       36       | 78186.908 |  8712110535.474 |
|       48       |  87556.46 | 11092675096.717 |
|       60       |  79272.03 |   9543997621.9  |
+----------------+-----------+-----------------+


In [154]:
df = timegpt.forecast(df=de_small_train, h=60, finetune_steps=50) # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("de_small_few200.csv")

на полном наборе прогноз даёт лучше

In [ ]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = de_small_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting_60:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('few-shot 200 illness'.center(len(table.field_names) * 11))
print(table)

       few-shot 200 illness      
+----------------+----------+-------------+
| Шагов прогноза |   MAE    |     MSE     |
+----------------+----------+-------------+
|       24       | 1470.311 | 8395972.454 |
|       36       | 1554.456 | 6937014.434 |
|       48       | 1374.993 | 5510307.978 |
|       60       | 1771.796 | 7041326.374 |
+----------------+----------+-------------+


In [155]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = de_small_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting_60:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('few-shot 200 illness'.center(len(table.field_names) * 11))
print(table)

       few-shot 200 illness      
+----------------+----------+--------------+
| Шагов прогноза |   MAE    |     MSE      |
+----------------+----------+--------------+
|       24       | 3286.235 | 16554268.008 |
|       36       | 2493.689 | 11449364.145 |
|       48       | 2473.08  | 10277568.608 |
|       60       | 2414.943 | 9550200.255  |
+----------------+----------+--------------+


In [156]:
%%time
df = timegpt.forecast(df=de_big_train, h=720, finetune_steps=200, model='timegpt-1-long-horizon') # предсказываю
predict = df['TimeGPT'] # вытаскиваю предсказания
predict.to_csv("de_big_few200.csv")

CPU times: user 194 ms, sys: 21.7 ms, total: 216 ms
Wall time: 24.5 s


In [157]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = de_big_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('few-shot 200 illness'.center(len(table.field_names) * 11))
print(table)

       few-shot 200 illness      
+----------------+----------+--------------+
| Шагов прогноза |   MAE    |     MSE      |
+----------------+----------+--------------+
|       24       | 878.724  | 1273005.413  |
|       96       | 3385.276 | 21958224.541 |
|      192       | 2323.418 | 12447990.44  |
|      336       | 2338.977 | 12256628.073 |
|      720       | 2614.634 | 15199761.023 |
+----------------+----------+--------------+


на полном наборе чуть лучше

In [ ]:
table = PrettyTable()
table.field_names = ["Шагов прогноза",  "MAE", "MSE"]
test = de_big_test['y']
# Перебираем разные шаги прогнозирования
for step in step_forecasting:
    # Вычисляем метрики
    MAE_value = mean_absolute_error(test.iloc[:step], predict[:step]).round(3)
    MSE_value = mean_squared_error(test.iloc[:step], predict[:step]).round(3)
    # Добавляем результаты в таблицу
    table.add_row([step, MAE_value, MSE_value])
print('few-shot 200 illness'.center(len(table.field_names) * 11))
print(table)

       few-shot 200 illness      
+----------------+----------+--------------+
| Шагов прогноза |   MAE    |     MSE      |
+----------------+----------+--------------+
|       24       | 1606.192 | 3530423.138  |
|       96       | 1664.431 | 3651187.867  |
|      192       | 1991.787 |  5300359.46  |
|      336       | 2313.559 | 6812958.804  |
|      720       | 2840.391 | 11975432.903 |
+----------------+----------+--------------+


в некоторых случаях метрики оставлялись, более лучшие, полученные на больших входных данных
в любом случае TimeGPT как не могла предсказать Illness, de_small, de_big так и не может